# Notions de Session, Contexte et Environnement

Initialisation de Vortex...

In [1]:
%load_ext ivortex
%vortex tmpcocoon

# [2019/09/03-17:19:24][vortex.sessions][_set_rundir:0152][WARNING]: Session <root> is changing its working directory </sauvegarde/vortex-dev/vortex/examples/notebooks/tutorial>
# [2019/09/03-17:19:24][vortex.sessions][_set_rundir:0155][INFO]: Session <root> set rundir </home/meunierlf/vortex-workdir/auto_cocoon_51yda7x0>


Vortex 1.6.2 loaded ( Tuesday 03. September 2019, at 17:19:23 )
The working directory is now: /home/meunierlf/vortex-workdir/auto_cocoon_51yda7x0/root


'/home/meunierlf/vortex-workdir/auto_cocoon_51yda7x0'

## Principe

À un moment donné il faut pouvoir :

  * Accéder à des données de configuration globales
  * Appeler des fonctions système (changement de répertoire, copie de fichiers, ...)
  * Avoir la liste des fichiers qui ont été récupérés/envoyés
  * Modifier des variables d'environnement, revenir en arrière, ...

Cela se traduit en Vortex par un arborescence d'objets :

* Une ou plusieurs *Session(s)* décrites chacune par un objet *Ticket* de session. Un *Ticket* de session a pour membres :

* Un objet *Glove* (_**GLO**bal **V**ersatile **E**nvironment_) qui contient des informations de configuration globales (nom d'utilisateur, profil de l'utilisateur, ...)
    * Un objet *System* qui permettra d'accéder à un grand nombre de fonctions système.
    * Un objet *Context* qui a pour membres :

        - Un objet *Sequence* qui conserve la mémoire des différentes 
          sections qui ont été créées lors des phases de 
          récupération et d'archivage des données
        - Un (ou plusieurs) objet(s) *Environment* qui permet(tent)
          d'interagir avec les variables d'environnement.
        - D'éventuels sous-contextes

Dans la description ci-dessus, on remarque qu'il peut y avoir plusieurs objets *Sessions*, *Contexts* ou *Environments*. Mais à un instant donné, un seul de ces objet sera "actif" et le lien hiérarchique sera conservé (le *Context* actif est forcément membre de la *Session* active).

**Exemple :** avoir plusieurs sous-*Context* peut avoir un intérêt si, dans un même script, on réalise deux tâches biens distinctes.

* On créera alors deux sous-contextes (*Ctx1* et *Ctx2*). 
* On active *Ctx1* et on commence à réaliser la première tâche dans 
  un sous-répertoire qui lui est propre : récupération de ressources, 
  modifications de l'environnement, etc.
* À un moment donné, on décide de réaliser la deuxième tâche. On active 
  alors *Ctx2* pour passer dans un répertoire dédié et repartir d'une 
  situation propre (*i.e.* l'environnement de *Ctx2* ne contient plus les 
  modifications faites dans *Ctx1* à l'occasion de la tâche précédente).
* On peut très bien décider de repartir travailler sur la première 
  tâche... Il suffit alors d'activer de nouveau *Ctx1* pour que 
  l'environnement de la première tâche soit restauré.
* ...

## En pratique...

L'utilisateur ou le développeur de tâches ne devrait pas trop se soucier de créer des *Sessions*, *Contexts*, ... car :

* Des objets *Session* et *Context* par défaut sont créés au chargement de Vortex : pour beaucoup de scripts cela suffit car on ne réalise qu'une seule tâche à la fois (c'est notamment le cas sous Olive).
* Dans le cas d'un job multi-tâches, ce travail de création des *Sessions* et *Contexts* est réalisé par le système "officiel" de création de jobs (présentation du dernier jour).

Il faut donc retenir qu'à un moment donné, il y une *Session*, un *Context* et un *Environment* actifs. La suite de la présentation se concentrera donc sur la récupération de ces objets "actifs" et sur leur utilisation.

Récupération de la session active (ou plus précisement de son *Ticket*) :

In [2]:
t = vortex.ticket()
# la session, en soi, n'est pas très intéressante mais elle donne accès
# à de nombreuses fonctionnalités

###  Le *Glove* de la session active

In [3]:
print(t.glove.idcard())

+ User     = meunierlf
+ Profile  = research
+ Vapp     = play
+ Vconf    = sandbox
+ Configrc = /home/meunierlf/.vortexrc


In [4]:
# un des rares points à personnaliser dans le glove est le vapp/vconf par défaut.
t.glove.vapp = 'arpege'
t.glove.vconf = '4dvarfr'
# (cela est pré-positionné par Olive ou par le système de création de jobs)

### L'objet *System* de la session active

In [5]:
print(t.sh)

<vortex.tools.systems.Linux34p object at 0x7ff02c88b400 | footprint=7>


Cet objet est d'un usage très courant, on y reviendra dans la suite de la présentation.

### L'objet *Context* actif

In [6]:
print(t.context)

Chaque *Context* a un chemin permettant de l'identifier dans la hiérarchie. Ici on travaille sur la session _"``root``"_ (créée par défaut au lancement de Vortex) et sur le contexte _"``root``"_ (créé automatiquement lors de la création de la session) :


In [7]:
print(t.context.path)

/root/root


### Les objets *Sequence* et *Environment* du contexte actif

In [8]:
print(t.context.sequence)

In [9]:
print(t.context.env)

<vortex.tools.env.Environment object at 0x7ff02c867be0 | including 105 variables>


Ces objets sont d'un usage très courant, on y reviendra dans la suite de la présentation.

## Le *System* : accéder aux fonctionnalités de l'OS

L'objet *System* donne accès à de nombreuses méthodes facilitant l'accès aux fonctionnalités du système d'exploitation.

L'objet *System* :

* Définit un certain nombre de méthodes propres à Vortex (cp, rm, touch, spawn, ...)
* Si une méthode n'est pas redéfinie dans l'objet *System*, il donne accès de façon transparente aux méthodes et modules mis à disposition par Python dans les modules standard ``os``, ``shutil`` et ``resource``.

In [10]:
# Nom du répertoire courant
current = t.sh.pwd()
# Changement de répertoire
print(t.sh.cd('/tmp'))
print(t.sh.pwd())
# Retour
print(t.sh.cd(current))

True
/tmp
True


In [11]:
files_list = t.sh.ls()
print(files_list)

['']


Les méthodes de cet objet sont très nombreuses, nous n'allons donc pas toutes les décrires ici. Pour une description détaillée, voir la documentation Sphinx du module **vortex.tools.systems** :

http://intra.cnrm.meteo.fr/algopy/sphinx/vortex/current/library/vortex/tools/systems.html

Comme indiqué précédemment, l'objet *System* donne accès à des méthodes ou modules présents dans les modules Python standard que sont ``os``, ``shutil`` et ``resource``. Par exemple, par ce biais, on utilise fréquement le module **os.path** de Python :

In [12]:
print('Exists ?', t.sh.path.exists('/dev/null'))
print('Basename:', t.sh.path.basename('/dev/null'))
print('Dirname:', t.sh.path.dirname('/dev/null'))

Exists ? True
Basename: null
Dirname: /dev


Pour la documentation de ces méthodes et modules, on se reportera à la documentation officielle de Python.

### Type des objets *System*

Les objets *System* ne sont pas forcement les mêmes d'une machine à l'autre. Ici :

In [13]:
type(t.sh)

vortex.tools.systems.Linux34p

La classe de l'objet *System* est choisie au moment du chargement de Vortex et de la création de la session par défaut. Ce choix s'effectue, via le package **footprints**, sur la base des données renvoyées par le module Python **platform** (type d'architecture, système d'exploitation, version de Python, ...).

Très souvent les objets ``Linux27`` ou ``Linux64p`` sont choisis mais il existe aussi des variantes pour MacOS ou naguère pour NEC-SX. Avoir des classes différentes selon les versions autorise d'avoir des implémentations différentes selon les types de systèmes, ce qui est très important dans le cadre d'une vision à long terme. En effet, un certain nombre de fonctions du module **os** de Python indiquent clairement que leurs implémentations ne sont pas portables d'une architecture à l'autre : il faut donc se réserver la possibilité de réaliser nos propres implémentations.

D'un point de vue pratique, les objets *System* des différentes architectures héritent tous de la classe **vortex.tools.systems.OSExtended** qui peut donc être vue comme une classe abstraite : toutes les méthodes présentes dans cette classe peuvent être utilisées quel que soit le système d'exploitation.

**Obligation dans le développement de Vortex :** l'objet *System* doit systématiquement être utilisé pour toute interaction avec le système d'exploitation, même si dans le cas de ``t.sh.path`` on retombe _actuellement_ sur une implémentation standard.

### Extension du shell (addons)

Il est possible d'ajouter, à la volée, de nouvelles fonctionnalités à l'objet *System*. De tels *Addons* peuvent être utilisés pour ajouter des méthodes spécifiques à tel ou tel modèle ou format de données (il s'agit de choses optionnelles).

Le plus simple est d'illustrer leur usage par un exemple :

In [14]:
class HelloAddon(vortex.tools.addons.Addon):  # Normalement défini dans le source de Vortex
    
    _footprint = dict(
        attr = dict(
            kind = dict(values = ['hello']),
        )
    )
    
    def say_hi(self):
        self.sh.title("Hello World !")

shhello = fp.proxy.addon(kind='hello', sh=sh)  # Chargement de l'Addon, en début de script

In [15]:
t.sh.say_hi()  # Voila !


=                                          HELLO WORLD !                                           =



**Principe** : Les méthodes publiques de l'objet *Addons* (*i.e.* celles dont le nom ne commence pas par ``_``) se retrouvent directement accessibles depuis l'objet *System*.

### Comportement différencié selon le format des données

Pour certains formats de données, les méthodes standard ne doivent pas être utilisées. Par exemple, Arpège utilise désormais des ``FA/LFI`` éclatés (format maison) qui sont en fait des répertoires contenant plusieurs fichiers associés à un fichier d'index ; pour ces données :

* un simple ``cp`` ou ``mv`` ne fonctionne pas (il faut appeler une commande spécifique)
* le ``diff`` ne fonctionne pas car les champs ne sont pas écrits dans un ordre déterministe
* il faut concaténer les différents fichiers du répertoire avant de les archiver (programme spécifique)

Pour satisfaire ce type de besoin, il est possible de personnaliser certaines méthodes en fonction du format. Il s'agit des méthodes : ``cp``, ``mv``, ``rm``, ``diff``, ``ftget``, ``rawftget``, ``ftput``, ``rawftput``, ``scpput`` et ``scpget``.

Lorsqu'un attribut (optionnel) ``fmt='format'`` est précisé lors de l'appel à l'une de ces méthodes (pour l'exemple, prenons ``cp``), on vérifie s'il existe dans l'objet *System* une méthode nommée ``format_cp``. Si tel est le cas, on l'appelle à la place de la fonction ``cp`` standard.

Les objets appelant ces méthodes doivent donc veiller à fournir des attributs ``fmt`` valides (sinon la méthode "standard" est utilisée). Dans Vortex, l'objet resource's *Handler* fournit systématiquement un format aux objets *Store* chargés de la récupération des données.

En pratique, les méthodes associées à des formats spécifiques sont définies dans des *Addons* afin que les utilisateurs qui n'en ont pas besoin ne soient pas "pollués" :

In [16]:
print('Avant:', hasattr(t.sh, 'lfi_cp'))
# la commande : t.sh.cp('origin', 'destination', fmt="lfi") fera un cp standard :-(
import vortex.tools.lfi
shlfi = fp.proxy.addon(kind='lfi', sh=t.sh)
print('Après:', hasattr(t.sh, 'lfi_cp'))
# la commande : t.sh.cp('origin', 'destination', fmt="lfi") fera désormais ce qu'il faut

Avant: False
Après: True


## L'objet *Sequence* du context actif

Récupérons d'abord quelques données afin que la démonstration prenne tout son sens.

In [17]:
r1 = toolbox.input(role='ArpegeNamelist', now=True, fatal=False, verbose=False, loglevel='warn',
                   # Resource
                   kind='namelist', model='arpege', source='something',
                   # Provider
                   genv='uget:cy42_op2.01fake@ugetdemo',
                   # Container
                   local='namelvoid')
r2 = toolbox.input(role='ArpegeNamelist', now=True, verbose=False, loglevel='warning',
                   # Resource
                   kind='namelist', model='arpege', source='namelistfc',
                   # Provider
                   genv='uget:cy42_op2.01fake@ugetdemo',
                   # Container
                   local='nameltest')
r2 = toolbox.input(role='RawGridpoint', now=True, verbose=False, loglevel='warning',
                   # Container
                   local='epytest_[geometry:area]_[term]', format='grib',
                   # Resource
                   kind='gridpoint', geometry='antil0025', origin='historic',
                   date='2016100100', term='0,3', cutoff='production', model='[vapp]',
                   nativefmt='[format]',
                   # Provider
                   experiment='OPER', block='forecast', namespace='vortex.multi.fr',
                   vapp='arome', vconf='antilles')

# [2019/09/03-17:19:25][vortex.tools.systems][cp:2211][ERROR]: Missing source /home/meunierlf/vortex-workdir/auto_cocoon_51yda7x0/root/cy42_op2.09lf.nam/something
# [2019/09/03-17:19:25][vortex.tools.systems][cp:2211][ERROR]: Missing source /home/meunierlf/vortex-workdir/auto_cocoon_51yda7x0/root/cy42_op2.09lf.nam/something
# [2019/09/03-17:19:25][vortex.data.stores][_verbose_log:0528][WARNING]: Multistore get uget://uget.multi.fr: none of the opened store succeeded.


In [18]:
seq = t.context.sequence

Récupérer la liste des *Sections* précédemment créées (*Attention :* on parle bien de *Sections*, pas de resource's *Handler*) :

In [19]:
seq.inputs()

<generator object Sequence.inputs at 0x7ff014f9ee60>

In [20]:
print('\n'.join(['{0.rh.container.abspath:s} ({0.role:s}, stage={0.stage})'.format(section)
                 for section in seq.inputs()]))

/home/meunierlf/vortex-workdir/auto_cocoon_51yda7x0/root/namelvoid (ArpegeNamelist, stage=void)
/home/meunierlf/vortex-workdir/auto_cocoon_51yda7x0/root/nameltest (ArpegeNamelist, stage=get)
/home/meunierlf/vortex-workdir/auto_cocoon_51yda7x0/root/epytest_ANTIL0025_00:00 (RawGridpoint, stage=get)
/home/meunierlf/vortex-workdir/auto_cocoon_51yda7x0/root/epytest_ANTIL0025_03:00 (RawGridpoint, stage=get)


L'attribut **_``stage``_** de la section permet de savoir quel est l'état de la ressource :

* *load* : La section a été créée mais rien de plus n'a été fait (now=False) ;
* *checked* : La ressource distante existe (méthode *check*) mais n'a pas été récupérée ;
* *void* : Erreur, la ressource demandée n'existe pas ;
* *get* : La ressource a été récupérée.

En général, on souhaite récupérer des *Sections* :

* Pour lesquelles le *get* a été fait et a fonctionné (``stage=get``)
* Pour un ``role`` bien précis (ou pour un ``kind`` donné)

La méthode **effective_inputs** est prévue à cet effet :


In [21]:
print('\n'.join(['{0.rh.container.abspath:s} ({0.role:s}, stage={0.stage})'.format(section)
                 for section in seq.effective_inputs(role='RawGridpoint')]))

/home/meunierlf/vortex-workdir/auto_cocoon_51yda7x0/root/epytest_ANTIL0025_00:00 (RawGridpoint, stage=get)
/home/meunierlf/vortex-workdir/auto_cocoon_51yda7x0/root/epytest_ANTIL0025_03:00 (RawGridpoint, stage=get)


In [22]:
print('\n'.join(['{0.rh.container.abspath:s} ({0.role:s}, stage={0.stage})'.format(section)
                 for section in seq.effective_inputs(role='ArpegeNamelist')]))
#NB: La section "void" n'apparait pas

/home/meunierlf/vortex-workdir/auto_cocoon_51yda7x0/root/nameltest (ArpegeNamelist, stage=get)


In [23]:
# Il est possible d'utiliser des expressions régulières en les compilant au préalable...
import re
regex = re.compile('(Arpege|Arome|Aladin)Namelist')
print('\n'.join(['{0.rh.container.abspath:s} ({0.role:s}, stage={0.stage})'.format(section)
                 for section in seq.effective_inputs(role=regex)]))

/home/meunierlf/vortex-workdir/auto_cocoon_51yda7x0/root/nameltest (ArpegeNamelist, stage=get)


On peut aussi se baser sur l'attribut ``kind`` de la ressource (à éviter autant que possible) :

In [24]:
print('\n'.join(['{0.rh.container.abspath:s} ({0.role:s}, stage={0.stage})'.format(section)
                 for section in seq.effective_inputs(kind='namelist')]))

/home/meunierlf/vortex-workdir/auto_cocoon_51yda7x0/root/nameltest (ArpegeNamelist, stage=get)


**Note:** Dans l'objet *Sequence*, des méthodes équivalentes existent pour les données de sortie (**outputs** et **effective_outputs**) mais sont beaucoup moins utilisées...

## L'objet *Environment* du contexte actif

On peut à tout moment récupérer l'objet *Environment* actif pour le contexte courant :

In [25]:
e = t.context.env
print(e)

<vortex.tools.env.Environment object at 0x7ff02c867be0 | including 105 variables>


Il est aussi possible d'utiliser le raccourci suivant :

In [26]:
e = t.env
print(e)

<vortex.tools.env.Environment object at 0x7ff02c867be0 | including 105 variables>


En première approche, on peut considérer que l'objet *Environment* s'utilise comme un dictionnaire Python (un peu amélioré)...

Les variables sont accessibles de plusieurs façons :

In [27]:
print(e['HOME'])
print(e.HOME)
print(e.home)

/home/meunierlf
/home/meunierlf
/home/meunierlf


Une variable manquante renvoie *None*...

In [28]:
print('FOO exists ?', 'FOO' in e)
print(e.FOO)

FOO exists ? False
None


Il y a quelques fonctions pratiques :

In [29]:
for var in [1, 'yes', 'ok', 'true']:
    e.FOO = var
    print("is True ?", e.true('FOO'))

is True ? True
is True ? True
is True ? True
is True ? True


Plus intéressant, il est possible de faire des changements temporaires :

In [30]:
e.FOO = 1
with e.delta_context(FOO=2, TOTO='is here'):
    print('foo={0.foo}, toto {0.toto}'.format(e))

# Les changements sont "oubliés" une fois que l'on sort du context manager:
print('foo={0.foo}, toto {0.toto}'.format(e))

foo=2, toto is here
foo=1, toto None


In [31]:
del e.FOO

Il existe quelques méthodes supplémentaires. Pour ceux qui sont intéressés :

* http://intra.cnrm.meteo.fr/algopy/sphinx/vortex/current/library/vortex/tools/env.html
* http://intra.cnrm.meteo.fr/algopy/sphinx/vortex/current/technical/environment.html (voir les exemples à la fin du document)

## Conclusion

Ces notions de *Session*, *Context*, *Sequence* et *Environment* peuvent faire peur, mais pour la plupart des utilisateurs et développeurs, il suffit de savoir comment récupérer les objets "actifs" (afin de les utiliser).

Pour ceux qui voudraient en savoir beaucoup plus sur ces questions :
http://intra.cnrm.meteo.fr/algopy/sphinx/vortex/current/technical/environment.html

Questions éventuelles : *vortex.support@meteo.fr*